In [1]:
from scipy.io import loadmat
from numpy import zeros as np_zeros, ones as np_ones
from numpy import vectorize as np_vectorize

from lib.stft import stft


# Test variables
num_rows, num_elements, num_beams = 1624, 65, 128
len_each_section = 16;
frac_overlap = 0.9;
padding = 16;
# winInfo = {@rectwin};
CHANDAT_FNAME = '/Users/zhanwenchen/Documents/projects/beam_nn/DNNs/test/scan_batteries/target_phantom_anechoic_cyst_2p5mm/target_1/chandat.mat'
chandat = loadmat(CHANDAT_FNAME)['chandat']

NEW_STFT_FNAME = '/Users/zhanwenchen/Documents/projects/beam_nn/DNNs/test/scan_batteries/target_phantom_anechoic_cyst_2p5mm/target_1/new_stft.mat'
new_stft_f = loadmat(NEW_STFT_FNAME)
new_stft_real = new_stft_f['new_stft_real']
new_stft_imag = new_stft_f['new_stft_imag']


In [2]:
chandat_stft = stft(chandat, len_each_section, frac_overlap, padding)
chandat_stft['origSigSize'] = [num_rows, num_elements, num_beams]


In [3]:
stft_data = chandat_stft['stft']

In [4]:
from numpy.linalg import norm
norm(stft_data)

1810982.5125408696

In [9]:
new_stft = new_stft_real + 1j*new_stft_imag


In [10]:
# create new and old stfts
from copy import copy

chandat_stft_new = copy(chandat_stft)
chandat_stft_new['stft'] = new_stft


In [11]:
shift_length = chandat_stft['shift_length']


In [12]:
len_each_section, num_rows_overlap, _, _ = chandat_stft['stft'].shape

In [13]:
# TODO: Isn't this just num_rows in the very beginning?
total_new_elements = (num_rows_overlap - 1) * shift_length + len_each_section

In [14]:
total_new_elements

1624

In [15]:
win_info = chandat_stft['win_info']

wVec = win_info(len_each_section)


In [16]:
wVecSq = wVec ** 2


In [17]:
from numpy import arange as np_arange


In [18]:
vecC = np_arange(1, num_rows_overlap*shift_length, step=shift_length)


In [19]:
from numpy import zeros as np_zeros

In [20]:
DlsArr = np_zeros((num_rows,))

In [21]:
# This was verified through and through
for j in vecC:
    tmpArr = np_arange(j-1, len_each_section+j-1)
    DlsArr[tmpArr] += wVecSq


In [22]:
DlsArrInv = 1/DlsArr

In [23]:
DlsArrInv

array([1. , 1. , 0.5, ..., 0.5, 1. , 1. ])

In [24]:
from numpy.linalg import norm

norm(DlsArr)

320.4746479832687

In [25]:
chandat_stft['stft'].dtype

dtype('complex128')

In [26]:
stft_data = chandat_stft['stft']

In [27]:
from numpy.fft import ifft as np_ifft
from numpy import sqrt as np_sqrt

invFT = np_sqrt(len_each_section)*np_ifft(chandat_stft['stft'], axis=0)


In [28]:
invFT_real = invFT.real

In [29]:
norm(invFT)

1810982.5125412922

In [30]:
from numpy import newaxis as np_newaxis

# invFT = invFT.*repmat(wVec,[1,size(invFT,2),size(invFT,3),size(invFT,4)]);
invFT *= wVec[:, np_newaxis, np_newaxis, np_newaxis]

In [31]:
yEst = np_zeros(chandat_stft['origSigSize'])
for index, j in enumerate(vecC):
    tmpArr = np_arange(j-1, len_each_section+j-1)
#     slice_norm = norm(invFT_real[:, index, :])
#     print('shape =', invFT_real[:, index, :].shape)
#     print('slice =', invFT_real[:, index, :])
#     print('slice_norm =', slice_norm)
#     print('after:', yEst[tmpArr, :])
    yEst[tmpArr, :] += invFT_real[:, index, :]


In [35]:
norm(yEst)

5070744.420360303

In [36]:
DlsArrInv.shape

(1624,)

In [39]:
# sigOut = yEst.*repmat(DlsArrInv,[1,size(yEst,2),size(yEst,3)]);
sigOut = yEst * DlsArrInv[:, np_newaxis, np_newaxis]


In [40]:
norm(sigOut)

657220.2353496825